## Stroke Work
Author: Daniel Maina Nderitu<br>
Project: MADIVA<br>
Purpose: Stroke modeling<br>
Notes:   We are comparing Poisson, robust Poisson, NB models, Random Forest, XGBoost, and LightGB.

#### Bootstrap cell

In [36]:
# =========================================================
# ML models for SHAP-compatible explainers
# =========================================================
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

import xgboost as xgb
import lightgbm as lgb

In [37]:
# =================== BOOTSTRAP CELL ===================
# Standard setup for all notebooks
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[0]  # assumes notebooks are in a subfolder
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# ========================================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

from src.config.variables import COVARIATES

# ========================================================
# Optional for warnings and nicer plots
import warnings
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

import sys
from pathlib import Path

# ========================================================
# 1️⃣ Ensure project root is in Python path
# Adjust this if your notebooks are nested deeper
PROJECT_ROOT = Path.cwd().parents[0]  # assumes notebooks are in a subfolder
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# ========================================================
# 2️⃣ Import helper to load paths
from src.utils.helpers import load_paths

# ========================================================
# 3️⃣ Load paths from config.yaml (works regardless of notebook location)
paths = load_paths()

# ========================================================
# 4️⃣ Optionally, print paths to confirm
for key, value in paths.items():
    print(f"{key}: {value}")

# ========================================================
# 5️⃣ Now you can use these paths in your notebook:
# Example:
DATA_DIR = paths['DATA_DIR']
OUT_DIR = paths['OUT_DIR']
FIG_DIR = paths['FIG_DIR']
MODEL_DIR = paths['MODEL_DIR']
# ========================================================

BASE_DIR: D:\APHRC\GoogleDrive_ii\stata_do_files\madiva\stroke_work
DATA_DIR: D:\APHRC\GoogleDrive_ii\stata_do_files\madiva\stroke_work\data
OUT_DIR: D:\APHRC\GoogleDrive_ii\stata_do_files\madiva\stroke_work\model_output
FIG_DIR: D:\APHRC\GoogleDrive_ii\stata_do_files\madiva\stroke_work\visualization
MODEL_DIR: D:\APHRC\GoogleDrive_ii\stata_do_files\madiva\stroke_work\model_output\statsmodels
NOTEBOOKS_DIR: D:\APHRC\GoogleDrive_ii\stata_do_files\madiva\stroke_work\notebooks
NOTEBOOKS_EXECUTED_DIR: D:\APHRC\GoogleDrive_ii\stata_do_files\madiva\stroke_work\notebooks_executed


### Import data - from previous step

In [38]:
# -----------------------------------------------------------------------------
# Loading saved data as pickle:
# -----------------------------------------------------------------------------
df = pd.read_pickle(OUT_DIR / "df_step06_processed.pkl")
X = pd.read_pickle(OUT_DIR / "X_step06_model_matrix.pkl")
y = pd.read_pickle(OUT_DIR / "y_step06_event.pkl")

#### prepare_pooled_data()

In [39]:
# =================================================================================  
# Prepare X, y, offset for pooled models
# =================================================================================  

def prepare_pooled_data(df, covariates, event_col='event', offset_col='offset'):
    """
    Prepare data for pooled regression models, handling 'Missing' string values.
    """
    
    # Select only covariates that exist in df (defensive)
    covariates_present = [c for c in covariates if c in df.columns]
    missing_covariates = set(covariates) - set(covariates_present)
    
    if missing_covariates:
        print(f"⚠️  Warning: Missing covariates: {missing_covariates}")
    print("✅ Covariates used:", covariates_present)
    
    # Create working copies
    X_pooled = df[covariates_present].copy()
    y = df[event_col].copy()
    offset = df[offset_col].copy()
    
    # --- First: Convert 'Missing' strings to NaN ---
    print("🔄 Converting 'Missing' strings to NaN...")
    
    for col in X_pooled.columns:
        # Check if column contains string 'Missing'
        if X_pooled[col].dtype == 'object':
            missing_count = (X_pooled[col] == 'Missing').sum()
            if missing_count > 0:
                print(f"   {col}: {missing_count} 'Missing' values → NaN")
                X_pooled[col] = X_pooled[col].replace('Missing', np.nan)
        
        # Also check for other missing representations
        other_missing_representations = ['missing', 'MISSING', 'Unknown', 'unknown', '']
        if X_pooled[col].dtype == 'object':
            for missing_val in other_missing_representations:
                if missing_val in X_pooled[col].values:
                    count = (X_pooled[col] == missing_val).sum()
                    if count > 0:
                        print(f"   {col}: {count} '{missing_val}' values → NaN")
                        X_pooled[col] = X_pooled[col].replace(missing_val, np.nan)
    
    # --- Now handle data types properly ---
    print("🔄 Converting data types...")
    
    for col in X_pooled.columns:
        if X_pooled[col].dtype == 'bool':
            # Convert pure boolean to int
            X_pooled[col] = X_pooled[col].astype(int)
            print(f"   {col}: bool → int")
            
        elif X_pooled[col].dtype == 'object':
            # Try to convert to numeric, which will handle the NaN values properly
            original_dtype = X_pooled[col].dtype
            X_pooled[col] = pd.to_numeric(X_pooled[col], errors='coerce')
            converted_nans = X_pooled[col].isna().sum()
            print(f"   {col}: {original_dtype} → numeric ({converted_nans} NaN values)")
            
        else:
            # Ensure numeric type
            original_dtype = X_pooled[col].dtype
            X_pooled[col] = pd.to_numeric(X_pooled[col], errors='coerce')
            if X_pooled[col].dtype != original_dtype:
                print(f"   {col}: {original_dtype} → {X_pooled[col].dtype}")
    
    # --- Check missingness percentage after conversion ---
    check_columns = covariates_present + [event_col, offset_col]
    missing_pct = (
        pd.concat([X_pooled, y.rename('event'), offset.rename('offset')], axis=1)
        .isna()
        .mean()
        .sort_values(ascending=False) * 100
    )
    
    print("\n🔎 Final percentage of missing values by variable:")
    for var, pct in missing_pct[missing_pct > 0].items():
        print(f"   {var}: {pct:.2f}%")
    
    if missing_pct.max() == 0:
        print("   No missing values found.")
    
    # --- Drop only rows where outcome or offset are missing or invalid ---
    valid_mask = (
        y.notna() & 
        np.isfinite(offset) & 
        (y >= 0)  # Assuming events should be non-negative
    )
    
    n_initial = len(df)
    n_final = valid_mask.sum()
    n_dropped = n_initial - n_final
    
    if n_dropped > 0:
        print(f"📊 Dropped {n_dropped} rows ({n_dropped/n_initial*100:.1f}%) due to missing/invalid outcomes or offsets")
    
    # Apply the mask
    X_pooled = X_pooled.loc[valid_mask]
    y_pooled = y.loc[valid_mask]
    offset_pooled = offset.loc[valid_mask]
    
    # --- Add constant term ---
    X_pooled_const = sm.add_constant(X_pooled, has_constant='add')
    
    print(f"\n✅ Final pooled dataset: {X_pooled_const.shape[0]} rows, {X_pooled_const.shape[1]} columns")
    print(f"   Features: {X_pooled_const.shape[1]-1} covariates + constant")
    
    return X_pooled_const, y_pooled, offset_pooled, covariates_present

# =================================================================================  
# Usage
# =================================================================================  
covariates_present = [c for c in COVARIATES if c in df.columns]
print("Covariates used:", covariates_present)

X_pooled_const, y_pooled, offset_pooled, covariates_used = prepare_pooled_data(df, covariates_present)

# =================================================================================  
# Verify the result
# =================================================================================  
print("\n🔍 Sample of processed X_pooled:")
print(X_pooled_const.head())
print(f"\nData types:\n{X_pooled_const.dtypes}")

Covariates used: ['sex_binary', 'alcohol_use', 'tobacco_use', 'hpt_status_derived', 'diab_status_derived', 'bmi_category_Overweight_Obese', 'hiv_status_derived', 'site_Nairobi']
✅ Covariates used: ['sex_binary', 'alcohol_use', 'tobacco_use', 'hpt_status_derived', 'diab_status_derived', 'bmi_category_Overweight_Obese', 'hiv_status_derived', 'site_Nairobi']
🔄 Converting 'Missing' strings to NaN...
🔄 Converting data types...

🔎 Final percentage of missing values by variable:
   tobacco_use: 0.60%

✅ Final pooled dataset: 28822 rows, 9 columns
   Features: 8 covariates + constant

🔍 Sample of processed X_pooled:
       const  sex_binary  alcohol_use  tobacco_use  hpt_status_derived  \
24278    1.0           1          888          0.0                   0   
24279    1.0           1            0          0.0                   1   
24280    1.0           0          888          0.0                   1   
24281    1.0           0            0          0.0                   1   
24282    1.0  

#### Poisson

In [40]:
df[COVARIATES + ['event', 'offset']].isna().sum()
# np.isinf(df[covariates + ['offset']]).sum()   # Check for infinity

sex_binary                         0
alcohol_use                        0
tobacco_use                      172
hpt_status_derived                 0
diab_status_derived                0
bmi_category_Overweight_Obese      0
hiv_status_derived                 0
site_Nairobi                       0
event                              0
offset                             0
dtype: int64

In [41]:
df_clean = df[COVARIATES + ['event', 'offset']].dropna() # dropping rows with missingness

In [42]:
import statsmodels.api as sm
import numpy as np

# =================================================================================  
X = sm.add_constant(df_clean[COVARIATES])
y = df_clean['event']

# =================================================================================  
# =================================================================================  
model_pois = sm.GLM(
    y,
    X,
    family=sm.families.Poisson(),
    offset=df_clean['offset']
).fit()

# =================================================================================  
# =================================================================================  
print(model_pois.summary())
print("\nIRR:")
print(np.exp(model_pois.params))

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  event   No. Observations:                28650
Model:                            GLM   Df Residuals:                    28641
Model Family:                 Poisson   Df Model:                            8
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.7710e+05
Date:                Mon, 02 Feb 2026   Deviance:                   3.5286e+05
Time:                        11:24:53   Pearson chi2:                 1.77e+05
No. Iterations:                     7   Pseudo R-squ. (CS):         -1.850e+05
Covariance Type:            nonrobust                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

##### Poisson Model Results

In [43]:
import numpy as np
import pandas as pd

# Print full model summary
print(model_pois.summary())

# -----------------------------------------------------------------------------
# Create a tidy summary DataFrame
# -----------------------------------------------------------------------------
results_df = pd.DataFrame({
    "Variable": model_pois.params.index,
    "Coef": model_pois.params.values,
    "StdErr": model_pois.bse,
    "z": model_pois.tvalues,
    "P>|z|": model_pois.pvalues,
    "CI_lower": model_pois.conf_int()[0],
    "CI_upper": model_pois.conf_int()[1]
})

# -----------------------------------------------------------------------------
# Add Incidence Rate Ratios (IRR)
# -----------------------------------------------------------------------------
results_df["IRR"] = np.exp(results_df["Coef"])
results_df["IRR_CI_lower"] = np.exp(results_df["CI_lower"])
results_df["IRR_CI_upper"] = np.exp(results_df["CI_upper"])

# print(results_df)
results_df.to_csv(OUT_DIR / "poisson_model_results_main.csv", index=False)

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  event   No. Observations:                28650
Model:                            GLM   Df Residuals:                    28641
Model Family:                 Poisson   Df Model:                            8
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.7710e+05
Date:                Mon, 02 Feb 2026   Deviance:                   3.5286e+05
Time:                        11:24:53   Pearson chi2:                 1.77e+05
No. Iterations:                     7   Pseudo R-squ. (CS):         -1.850e+05
Covariance Type:            nonrobust                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

#### Robust Poisson

In [44]:
# Fit Robust Poisson (same coefficients, larger SEs)
model_robust = sm.GLM(y, X, family=sm.families.Poisson(), offset=df_clean["offset"]).fit(cov_type='HC0')

#### Negative Binomial

In [45]:
model_nb = sm.GLM(y, X, family=sm.families.NegativeBinomial(), offset=df_clean["offset"]).fit()

#### Model comparison

In [46]:
summary = summary_col(
    results=[model_pois, model_robust, model_nb],
    model_names=['Poisson', 'Robust Poisson', 'NegBinomial'],
    stars=True,
    float_format='%0.3f',
    info_dict={'N':lambda x: f"{int(x.nobs)}"}
)
print(summary)


                               Poisson  Robust Poisson NegBinomial
------------------------------------------------------------------
const                         -0.000*** -0.000         -5.716***  
                              (0.000)   (0.018)        (0.099)    
sex_binary                    0.033***  0.033**        0.032      
                              (0.000)   (0.015)        (0.083)    
alcohol_use                   -0.000*** -0.000***      -0.000***  
                              (0.000)   (0.000)        (0.000)    
tobacco_use                   -0.001*** -0.001***      -0.001*    
                              (0.000)   (0.000)        (0.000)    
hpt_status_derived            0.246***  0.246***       0.278***   
                              (0.000)   (0.014)        (0.079)    
diab_status_derived           -0.254*** -0.254***      -0.280***  
                              (0.000)   (0.014)        (0.079)    
bmi_category_Overweight_Obese 0.070***  0.070***       0.059 

#### IRRs

In [47]:
def extract_irrs(model):
    df_irr = pd.DataFrame({
        "Variable": model.params.index,
        "Coef": model.params.values,
        "StdErr": model.bse,
        "z": model.tvalues,
        "P>|z|": model.pvalues,
        "CI_lower": model.conf_int()[0],
        "CI_upper": model.conf_int()[1]
    })
    df_irr["IRR"] = np.exp(df_irr["Coef"])
    df_irr["IRR_CI_lower"] = np.exp(df_irr["CI_lower"])
    df_irr["IRR_CI_upper"] = np.exp(df_irr["CI_upper"])
    # Add significance stars
    df_irr["sig"] = df_irr["P>|z|"].apply(lambda p: 
                                  "***" if p < 0.001 else 
                                  "**" if p < 0.01 else 
                                  "*" if p < 0.05 else "")
    return df_irr

# =================================================================================  
# =================================================================================  

results_pois = extract_irrs(model_pois)
results_robust = extract_irrs(model_robust)
results_nb = extract_irrs(model_nb)

# =================================================================================  
# --- 4️⃣ Combine results for export ---
# =================================================================================  
all_results = pd.concat([
    results_pois.assign(Model='Poisson'),
    results_robust.assign(Model='Robust Poisson'),
    results_nb.assign(Model='NegBinomial')
])

# =================================================================================  
# Save table to Excel
# =================================================================================  
all_results.to_excel(OUT_DIR / "stroke_model_results_comparison_main.xlsx", index=False)
print("✅ Model comparison results saved to Excel.")

✅ Model comparison results saved to Excel.


#### Overdispersion

In [48]:
dispersion = model_pois.deviance / model_pois.df_resid
print("Dispersion parameter:", dispersion)

if dispersion > 1.5:
    print("⚠️ Data likely overdispersed — Negative Binomial model may be more appropriate.")
else:
    print("✅ Poisson model dispersion acceptable.")

Dispersion parameter: 12.320018338020425
⚠️ Data likely overdispersed — Negative Binomial model may be more appropriate.


### Train/Test Split (shared by all ML models)

In [49]:
# =========================================================
# Train / Test split
# =========================================================
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

print(X_train.shape, X_test.shape)

(21487, 9) (7163, 9)


#### Random Forest Model (SHAP-friendly)
##### Train RF

In [50]:
# =========================================================
# Random Forest
# =========================================================
rf_model = RandomForestClassifier(
    n_estimators=400,
    max_depth=6,
    min_samples_leaf=50,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

,n_estimators,400
,criterion,'gini'
,max_depth,6
,min_samples_split,2
,min_samples_leaf,50
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


##### Evaluate RF

In [51]:
rf_pred = rf_model.predict_proba(X_test)[:, 1]

rf_auc = roc_auc_score(y_test, rf_pred)
print("Random Forest AUC:", rf_auc)

Random Forest AUC: 0.6639138500289322


#### XGBoost Model
##### Train xgb

In [52]:
# =========================================================
# XGBoost
# =========================================================
xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    random_state=42
)

xgb_model.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


##### Evaluate xgb

In [53]:
xgb_pred = xgb_model.predict_proba(X_test)[:, 1]

xgb_auc = roc_auc_score(y_test, xgb_pred)
print("XGBoost AUC:", xgb_auc)

XGBoost AUC: 0.6431762483406515


#### LightGBM Model
##### Train lgb

In [54]:
lgb_model = lgb.LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)

lgb_model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 503, number of negative: 20984
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000660 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 25
[LightGBM] [Info] Number of data points in the train set: 21487, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.023410 -> initscore=-3.730925
[LightGBM] [Info] Start training from score -3.730925


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.05
,n_estimators,500
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


##### Evaluate lgb

In [55]:
lgb_pred = lgb_model.predict_proba(X_test)[:, 1]

print("LightGBM AUC:", roc_auc_score(y_test, lgb_pred))

LightGBM AUC: 0.6106240852309472


#### Model Comparison Table

In [56]:
model_compare = pd.DataFrame({
    "model": [
        "Poisson",
        "Negative Binomial",
        "Random Forest",
        "XGBoost"
    ],
    "AUC": [
        None,  # fill if you computed
        None,
        rf_auc,
        xgb_auc
    ]
})

model_compare

,model,AUC
0,Poisson,NaN
1,Negative Binomial,NaN
2,Random Forest,0.663914
3,XGBoost,0.643176


#### End - Saving Models and Data

In [57]:
# Saved as pickle (faster for large data, preserves types)
df.to_pickle(OUT_DIR / "df_step07_processed.pkl")
results_df.to_pickle(OUT_DIR / "df_step07_results_df.pkl")
all_results.to_pickle(OUT_DIR / "df_step07_all_results.pkl")

X.to_pickle(OUT_DIR / "X_step07_model_matrix.pkl")
y.to_pickle(OUT_DIR / "y_step07_event.pkl")

results_pois.to_pickle(OUT_DIR / "results_pois.pkl") 
results_robust.to_pickle(OUT_DIR / "results_robust.pkl") 
results_nb.to_pickle(OUT_DIR / "results_nb.pkl")

# =================================================================================  
# Saving models
# =================================================================================  
import pickle

with open(MODEL_DIR / "model_pois.pkl", "wb") as f:
    pickle.dump(model_pois, f)

with open(MODEL_DIR / "model_robust.pkl", "wb") as f:
    pickle.dump(model_robust, f)

with open(MODEL_DIR / "model_nb.pkl", "wb") as f:
    pickle.dump(model_nb, f)

# Tree based models
with open(MODEL_DIR / "rf_model.pkl", "wb") as f:
    pickle.dump(rf_model, f)    

with open(MODEL_DIR / "xgb_model.pkl", "wb") as f:
    pickle.dump(xgb_model, f)

with open(MODEL_DIR / "lgb_model.pkl", "wb") as f:
    pickle.dump(lgb_model, f)
